In [1]:
# import required libraries
import pandas as pd
import numpy as np
import os

In [2]:
def trim_dataframe(df):
    # Create a boolean mask where 'Subtotal' is present
    mask = df.apply(lambda row: row.astype(str).str.contains('Subtotal')).any(axis=1)
    
    # Find the index of the first occurrence of 'Subtotal'
    if mask.any():
        subtotal_index = mask.idxmax()  # Get the index of the first True value
        
        # Return the DataFrame up to the row before 'Subtotal'
        trimmed_df = df.iloc[:subtotal_index-1]
    else:
        # If 'Subtotal' is not found, return the original DataFrame
        trimmed_df = df
    
    return trimmed_df

In [3]:
# directory containing HTML files
html_dir = '../data/html_data'

# List to store DataFrames
html_dfs = []

# iterate over files in the directory
for file in os.listdir(html_dir):
    if file.endswith('.html'):
        # Construct the full path to the HTML file
        file_path = os.path.join(html_dir, file)
        
        # Read HTML tables from the file
        tables = pd.read_html(file_path)
        
        # Append the DataFrames to the list
        html_dfs.extend(tables)
        
print('Number of table extracted:', len(html_dfs))

Number of table extracted: 74


In [4]:
processed_dfs = []
for df in html_dfs:
    trimmed_df = trim_dataframe(df.iloc[1:,:])
    processed_dfs.append(trimmed_df.iloc[:,:2])

In [5]:
cont = []
for df in processed_dfs:
    if df.shape[0] == 0:
        continue
    else:
        if '[' in df.iloc[0, 0]:
            cont.append(df)

In [6]:
merged_data = pd.concat(cont, axis=0)
merged_data.columns = ['Description', 'Number_Samples']

In [7]:
# extract the order ID and order date from the description column
merged_data['Order_ID'] = merged_data['Description'].apply(lambda x: x.split(' ')[0]).str.strip('[').str.strip(']')

merged_data['Order_Date'] = merged_data['Order_ID'].apply(lambda x: x.split('F')[0])
merged_data['Order_Date'] = pd.to_datetime(merged_data['Order_Date'], format='%y%m%d')
merged_data['Order_Date'] = merged_data['Order_Date'].dt.strftime('%Y-%m-%d')

In [8]:
# subset the dataframe with selected column, sort by date and export into Excel file
invoice = merged_data[['Order_ID','Order_Date','Number_Samples']]
sorted_invoice = invoice.sort_values(by='Order_Date')
sorted_invoice.to_excel('../data/macrogen_invoice.xlsx', index=False)

In [183]:
html = pd.read_html('../data/html_data/old/ISC200228229.html')
print(len(html))

2


In [184]:
html[0]

,0,1,2,3,4,5
0,Order No.,Service description,Q'ty,Unit,Unit price,Amount (SGD)
1,200131FN-090,Plasmid Sequencing,4,Reaction,4.6,18.4
2,200131FN-088,PCR product Sequencing,11,Reaction,4.6,50.6
3,200131FN-094,PCR product Sequencing,8,Reaction,4.6,36.8
4,200203FN-038,PCR product Sequencing,3,Reaction,4.6,13.8
5,200203FN-037,Plasmid Sequencing,4,Reaction,4.6,18.4
6,200204FN-048,PCR product Sequencing,12,Reaction,4.6,55.2
7,200205FN-086,PCR product Sequencing,12,Reaction,4.6,55.2
8,200206FN-079,PCR product Sequencing,4,Reaction,4.6,18.4
9,200207FN-076,PCR product Sequencing,8,Reaction,4.6,36.8


In [185]:
html[1]

,0,1,2,3,4,5
0,200211FN-126,PCR product Sequencing,4,Reaction,4.6,18.4
1,200212FN-067,Plasmid Sequencing,2,Reaction,4.6,9.2
2,200214FN-049,PCR product Sequencing,8,Reaction,4.6,36.8
3,200214FN-057,PCR product Sequencing,12,Reaction,4.6,55.2
4,200217FN-012,PCR product Sequencing,4,Reaction,4.6,18.4
5,200217FN-105,PCR product Sequencing,12,Reaction,4.6,55.2
6,200218FN-052,PCR product Sequencing,1,Reaction,4.6,4.6
7,200218FN-084,PCR product Sequencing,6,Reaction,4.6,27.6
8,200220FN-064,PCR product Sequencing,4,Reaction,4.6,18.4
9,200220FN-107,PCR product Sequencing,26,Reaction,4.6,119.6


In [188]:
merged = pd.concat([html[0], html[1]], axis=0)
merged

,0,1,2,3,4,5
0,Order No.,Service description,Q'ty,Unit,Unit price,Amount (SGD)
1,200131FN-090,Plasmid Sequencing,4,Reaction,4.6,18.4
2,200131FN-088,PCR product Sequencing,11,Reaction,4.6,50.6
3,200131FN-094,PCR product Sequencing,8,Reaction,4.6,36.8
4,200203FN-038,PCR product Sequencing,3,Reaction,4.6,13.8
5,200203FN-037,Plasmid Sequencing,4,Reaction,4.6,18.4
6,200204FN-048,PCR product Sequencing,12,Reaction,4.6,55.2
7,200205FN-086,PCR product Sequencing,12,Reaction,4.6,55.2
8,200206FN-079,PCR product Sequencing,4,Reaction,4.6,18.4
9,200207FN-076,PCR product Sequencing,8,Reaction,4.6,36.8


In [189]:
merged = merged.iloc[1:-1, :].copy()

In [190]:
merged.columns = ['Order_ID', 'Description', 'Number_Samples', 'UOM', 'Unit_Price', 'Total']

In [191]:
merged_subset = merged[~merged['Order_ID'].isna()].copy()

In [192]:
merged_subset

,Order_ID,Description,Number_Samples,UOM,Unit_Price,Total
1,200131FN-090,Plasmid Sequencing,4,Reaction,4.6,18.4
2,200131FN-088,PCR product Sequencing,11,Reaction,4.6,50.6
3,200131FN-094,PCR product Sequencing,8,Reaction,4.6,36.8
4,200203FN-038,PCR product Sequencing,3,Reaction,4.6,13.8
5,200203FN-037,Plasmid Sequencing,4,Reaction,4.6,18.4
6,200204FN-048,PCR product Sequencing,12,Reaction,4.6,55.2
7,200205FN-086,PCR product Sequencing,12,Reaction,4.6,55.2
8,200206FN-079,PCR product Sequencing,4,Reaction,4.6,18.4
9,200207FN-076,PCR product Sequencing,8,Reaction,4.6,36.8
10,200210FN-027,PCR product Sequencing,4,Reaction,4.6,18.4


In [193]:
# extract the order ID and order date from the description column
#merged_subset['Order_ID'] = merged_subset['Description'].apply(lambda x: x.split(' ')[0]).str.strip('[').str.strip(']')

merged_subset['Order_Date'] = merged_subset['Order_ID'].apply(lambda x: x.split('F')[0])
merged_subset['Order_Date'] = pd.to_datetime(merged_subset['Order_Date'], format='%y%m%d')
merged_subset['Order_Date'] = merged_subset['Order_Date'].dt.strftime('%Y-%m-%d')

In [194]:
merged_subset

,Order_ID,Description,Number_Samples,UOM,Unit_Price,Total,Order_Date
1,200131FN-090,Plasmid Sequencing,4,Reaction,4.6,18.4,2020-01-31
2,200131FN-088,PCR product Sequencing,11,Reaction,4.6,50.6,2020-01-31
3,200131FN-094,PCR product Sequencing,8,Reaction,4.6,36.8,2020-01-31
4,200203FN-038,PCR product Sequencing,3,Reaction,4.6,13.8,2020-02-03
5,200203FN-037,Plasmid Sequencing,4,Reaction,4.6,18.4,2020-02-03
6,200204FN-048,PCR product Sequencing,12,Reaction,4.6,55.2,2020-02-04
7,200205FN-086,PCR product Sequencing,12,Reaction,4.6,55.2,2020-02-05
8,200206FN-079,PCR product Sequencing,4,Reaction,4.6,18.4,2020-02-06
9,200207FN-076,PCR product Sequencing,8,Reaction,4.6,36.8,2020-02-07
10,200210FN-027,PCR product Sequencing,4,Reaction,4.6,18.4,2020-02-10


In [195]:
merged_subset2 = merged_subset[['Order_ID', 'Order_Date', 'Number_Samples']]

In [196]:
merged_subset2.to_excel('../data/manualinsert.xlsx', index=False)